In [2]:
import pandas as pd
import numpy as np

In [3]:
df_net= pd.read_csv('transformacion_netflix.csv', delimiter=',', encoding='utf-8')
df_dis= pd.read_csv('transformacion_disney.csv', delimiter=',', encoding='utf-8')
df_hulu= pd.read_csv('transformacion_hulu.csv', delimiter=',', encoding='utf-8')
df_amazon= pd.read_csv('transformacion_amazon.csv', delimiter=',', encoding='utf-8')
df_ratings=pd.read_csv('transformacion_ratings.csv', delimiter=',', encoding='utf-8')


In [4]:
#creo un archivo gral
df_plataformagral = pd.concat([df_net,df_hulu,df_amazon,df_dis])

In [5]:
#lo guardo
df_plataformagral.to_csv('totalplataformas.csv', index=False)

Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN

In [6]:

def get_max_duration(year:int=None,platform:str=None,duration_type:str=None):
    
    #declaro parametros
    if platform == 'netflix':
        inicioId = 'n'
    elif platform == 'hulu':
        inicioId = 'h'
    elif platform == 'disney':
        inicioId = 'd'
    elif platform == 'amazon':
        inicioId = 'a'
    elif platform is None:
        inicioId = None
    else:
        return 'no se encuentra plataforma, ingrear una de las opciones: netflix, hulu, disney o amazon'
    

    if duration_type == 'season':
        durationtype= 'season'
    elif duration_type == 'film':
        durationtype = 'min'
    else:
        return 'ingresar season o film en minuscula' 

  
    df_plataformagral=  pd.read_csv('totalplataformas.csv', delimiter=',', encoding='utf-8')

    #aplico filtro para posibles escenarios

    if year and platform and  duration_type  is not None: #si se ingresan todos los parametros

        filtro1= df_plataformagral[df_plataformagral.id.str.contains(inicioId) #filtro por primer letra de plataforma
                            & (df_plataformagral.release_year==year)  #filtro por el año
                            & (df_plataformagral.duration_type==durationtype) ] #filtro por la duracion
        max = filtro1['duration_int'].max() #guardo el valor maximo
        filtro2 = filtro1.query("duration_int==@max") #consulto la duracion de ese valor
        pelicula = filtro2[['id','title', 'duration_int', 'duration_type']] #guardo lo que quiero que retorne

        if  not pelicula.empty: #si segun los datos ingresados el dataframe no esta vacio

            return pelicula
    
    elif year is None: #si no se pasa el parametro year
        
        if inicioId is None : #si parametro year y inicoId no se ingresan
            
            filtro1= df_plataformagral[(df_plataformagral.duration_type==durationtype) ] #filtro por la duracion
            max = filtro1['duration_int'].max()
            filtro2 = filtro1.query("duration_int==@max")
            pelicula = filtro2[['title', 'duration_int', 'duration_type']]
            return pelicula        
        
        filtro1= df_plataformagral[df_plataformagral['id'].str.contains(inicioId) #filtro por primer letra de plataforma
                            & (df_plataformagral.duration_type==durationtype) ] #filtro por la duracion
        max = filtro1['duration_int'].max()
        filtro2 = filtro1.query("duration_int==@max")
        pelicula = filtro2[['title', 'duration_int', 'duration_type']]
        return pelicula
                   
    elif inicioId is None : #si inicioid no se ingresa
        if durationtype is None: #si parametro  inicoId y durationtype no se ingresan

            filtro1= df_plataformagral[(df_plataformagral.release_year==year)]#filtro por año                    
            max = filtro1['duration_int'].max()
            filtro2 = filtro1.query("duration_int==@max")
            pelicula = filtro2[['id','title', 'duration_int', 'duration_type']]
            return pelicula    
     
        filtro1= df_plataformagral[(df_plataformagral.release_year==year) #filtro por año
                                   & (df_plataformagral.duration_type==durationtype) ] #filtro por la duracion
        max = filtro1['duration_int'].max()
        filtro2 = filtro1.query("duration_int==@max")
        pelicula = filtro2[['id','title', 'duration_int', 'duration_type']]   
        return pelicula 


In [8]:
get_max_duration(None,'hulu', 'film')

,title,duration_int,duration_type
9104,stephen king's it,192.0,min


Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año 

In [9]:
df_ratings= pd.read_csv('transformacion_ratings.csv', delimiter=',', encoding='utf-8')


In [10]:
df_ratings['fecha']= pd.to_datetime(df_ratings['fecha']) #necesito separar solo en año en nueva columna
df_ratings['year']= df_ratings['fecha'].dt.year
df_ratings.head(1)

,userId,rating,timestamp,id,fecha,score,year
0,1,1.0,1425941529,as680,2015-03-09 22:52:09,3.549127,2015


In [13]:
def get_score_count(platform, scored, year):  

    import datetime

    #declaro parametros
    if platform == 'netflix':
        plataforma = 'n'
    elif platform == 'hulu':
        plataforma = 'h'
    elif platform == 'disney':
        plataforma = 'd'
    elif platform == 'amazon':
        plataforma = 'a'
    else:
        return 'ingresar una de las opciones: netflix, hulu, amazon o disney'
    

    df_ratings = pd.read_csv(r'transformacion_ratings.csv', delimiter=',', encoding='utf-8')
    
    df_ratings['fecha']= pd.to_datetime(df_ratings['fecha']) #modifico el tipo de objeto
    df_ratings['year']= df_ratings['fecha'].dt.year #agrego columna de año
    consulta_ratings= df_ratings[(df_ratings.year==year) #busco segun año
                            & (df_ratings.id.str.contains(plataforma))] #filtro por id

    consulta_score = consulta_ratings[consulta_ratings.score> scored ] #filtro por score>a lo ingresado

    if consulta_score.empty :
        return 'ingresar scored mayor a 3.3 y menor a 3.8, o el año ingresado no tiene registro'
    else:
       return 'cantidad de peliculas',consulta_score.id.count()

In [14]:
get_score_count('netflix', 3.7, 2015)

('cantidad de peliculas', 38)

Cantidad de películas por plataforma con filtro de PLATAFORMA

In [15]:
def get_count_platform(platform): 

    #Declaro parametros
    if platform == 'netflix':
        plataforma = 'n'
    elif platform == 'hulu':
        plataforma= 'h'
    elif platform == 'disney':
        plataforma= 'd'
    elif platform == 'amazon':
        plataforma= 'a'
    else:
        return print('ingrese plataforma: amazon, disney, hulu o netflix')

    df_plataformagral =  pd.read_csv('totalplataformas.csv', delimiter=',', encoding='utf-8')

    consulta_plataforma= df_plataformagral[(df_plataformagral.id.str.contains(plataforma))#filtro por plataforma
                                           & (df_plataformagral.duration_type=='min')]#filtro por duration
    conteo_movies= consulta_plataforma.title.count()#cuento
    return conteo_movies

In [16]:
get_count_platform('netflix')

6128

Actor que más se repite según plataforma y año

In [17]:
def get_actor(platform, year): 

    #declaro parametros
    if platform == 'netflix':
        plataforma= 'n'
    elif platform == 'disney':
        plataforma= 'd'
    elif platform == 'hulu':
        plataforma= 'h'
    elif platform == 'amazon':
        plataforma= 'a'
    else:
        return 'ingresar plataforma valida: amazon, disney, hulu o netflix'

    
    df_plataformagral =  pd.read_csv('totalplataformas.csv', delimiter=',', encoding='utf-8')   

    filtroplat = df_plataformagral[(df_plataformagral.id.str.contains(plataforma))
                                   & (df_plataformagral.release_year==year)] #filtro por año y plataforma

    if filtroplat.empty: #si devuelve df vacio
        
        return 'el año no se encuentra en el dataset'
    else:
        lista= filtroplat['cast'] #guardo en nueva variable los actores por peliculas
        newlista = [x for x in lista if pd.isnull(x) == False]#elimino flotantes

        lista_cast=[]
        for i in newlista: #itero para separar actores en diferentes str
            separador= i.split(sep=',')
            for j in separador: #itero para eliminar espacios en cada nuevo str y agregar a lista
                    valores=j.strip()
                    lista_cast.append(valores) #agrego valores a lista_cast
        df_lista_cast= pd.Series(lista_cast) #transformo a dataframe
        return ('El actor mas repetido es:', df_lista_cast.value_counts().idxmax()) #imprimo copnsulta del str mas repetido

In [18]:
get_actor('netflix', 2015)

('El actor mas repetido es:', 'james franco')